<a href="https://colab.research.google.com/github/syoungk7/leetcode_mysolution/blob/main/Leetcode_sql_mysolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Selcet

In [ ]:
-- 1757. Recyclable and Low Fat Products: find the ids of products that are both low fat and recyclable.
select product_id
from Products
where low_fats = 'Y' and recyclable = 'Y'

In [ ]:
-- 584. Find Customer Referee: Find the names of the customer that are not referred by the customer with id = 2.
select name
from Customer
where referee_id is null or referee_id != '2'

In [ ]:
-- 595. Big Countries: find the name, population, and area of the big countries.
select name, population, area
from World
where area >= 3000000 or population >= 25000000

In [ ]:
-- 1148. Article Views I: find all the authors that viewed at least one of their own articles.
select author_id as id
from Views
where author_id = viewer_id
group by author_id
order by id
--# distinct

In [ ]:
-- 1683. Invalid Tweets: find the IDs of the invalid tweets.
select tweet_id
from Tweets
Where char_length(content) > 12
--# length -> bytes

In [ ]:
-- A 1821. Find Customers With Positive Revenue this Year: report the customers with postive revenue in the year 2021.
SELECT customer_id
FROM Customers
WHERE year = 2021
GROUP BY customer_id, year
HAVING SUM(revenue) > 0

In [ ]:
-- A 183. Customers Who Never Order: find all customers who never order anything.
SELECT c.name AS Customers
FROM Customers c
LEFT JOIN Orders o ON (o.customerId = c.id)
WHERE o.customerId IS NULL

In [ ]:
-- A 1873. Calculate Special Bonus: calculate the bonus of each employee.
Select employee_id,
        If(employee_id % 2 = 1 and name not Like 'M%', salary, 0) as bonus
From Employees
Order by employee_id

In [ ]:
-- AM 1398. Customers Who Bought Products A and B but Not C:
-- report the customer_id and customer_name of customers who bought products "A", "B" but did not buy the product "C"
WITH Counting AS (SELECT t.customer_id,
                    CASE t.product_name
                        WHEN 'A' THEN 1
                        WHEN 'B' THEN 1
                        WHEN 'C' THEN -1
                        ELSE 0
                    END AS tmp_count
                  FROM (SELECT customer_id, product_name
                        FROM Orders
                        GROUP BY customer_id, product_name) t)

SELECT o.customer_id, c.customer_name
FROM Counting o, Customers c
WHERE o.customer_id = c.customer_id
GROUP BY o.customer_id
HAVING SUM(o.tmp_count) = 2

In [ ]:
-- AM 1112. Highest Grade For Each Student:
-- find the highest grade with its corresponding course for each student. In case of a tie, you should find the course with the smallest course_id.
WITH ranking AS (
    SELECT student_id, course_id, grade,
        RANK() OVER (PARTITION BY student_id ORDER BY grade DESC, course_id) AS ranks
    FROM Enrollments
    ORDER BY student_id)

SELECT student_id, course_id, grade
FROM ranking
WHERE ranks = 1

--# having min(course_id)

## 2. Basic Joins

In [ ]:
-- 1378. Replace Employee ID With The Unique Identifier: show the unique ID of each user, If a user does not have a unique ID replace just show null.
select u.unique_id, e.name
from Employees e
left join EmployeeUNI u on e.id = u.id

In [ ]:
-- 1068. Product Sales Analysis I: report the product_name, year, and price for each sale_id in the Sales table.
select p.product_name, s.year, s.price
from Sales s
right join Product p on s.product_id = p.product_id
where s.price is not Null

--# select p.product_name, s.year, s.price
--# from Sales s, Product p
--# where s.product_id = p.product_id

--# inner join

In [ ]:
-- 1581. Customer Who Visited but Did Not Make Any Transactions:
-- find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.
select v.customer_id, count(v.customer_id) as count_no_trans
from Visits v
left join Transactions t on v.visit_id = t.visit_id
where t.amount is Null
group by v.customer_id

--# select customer_id, count(customer_id) as count_no_trans
--# from Visits
--# where visit_id not in (select visit_id from Transactions)
--# group by customer_id
--# order by count_no_trans Desc

In [ ]:
-- 197. Rising Temperature: find all dates' Id with higher temperatures compared to its previous dates (yesterday).
select w.id
from Weather w, Weather e
where timestampdiff(day, e.recordDate, w.recordDate) = 1 and w.temperature - e.temperature > 0

--# With tmp_weather (id, recordDate, temperature, prev_day, prev_tem)
--# As (Select id, recordDate, temperature,
--#     Lag(recordDate) over (Order by recordDate) prev_day,
--#     Lag(temperature) over (Order by recordDate) prev_tem
--#     From Weather
--#     )
--# Select id
--# From tmp_weather
--# Where temperature - prev_tem > 0 and TIMESTAMPDIFF(Day, prev_day, recordDate) = 1

--# join
--# select w.id
--# from Weather w
--# Join Weather e on TIMESTAMPDIFF(Day, e.recordDate, w.recordDate) = 1 and w.temperature - e.temperature > 0

In [ ]:
-- 1661. Average Time of Process per Machine: find the average time each machine takes to complete a process.
select a.machine_id, round(sum((c.timestamp - a.timestamp))/count(a.process_id), 3) as processing_time
from Activity a
join Activity c on a.machine_id = c.machine_id and a.process_id = c.process_id and a.activity_type = 'start' and c.activity_type = 'end'
group by a.machine_id

In [ ]:
-- 577. Employee Bonus: report the name and bonus amount of each employee with a bonus less than 1000.
select e.name, b.bonus
from Employee e
left join Bonus b on e.empId = b.empId
where b.bonus < 1000 or b.bonus is Null

In [ ]:
-- 1280. Students and Examinations: find the number of times each student attended each exam.
select s.student_id, s.student_name, s.subject_name, count(e.student_id) as attended_exams
from (select student_id, student_name, subject_name
      from Students, Subjects) s
left join Examinations e on e.student_id = s.student_id and e.subject_name = s.subject_name
group by s.student_id, s.subject_name
order by s.student_id, s.subject_name

--# with s as (select student_id, student_name, subject_name from Students, Subjects)

In [ ]:
-- M 570. Managers with at Least 5 Direct Reports: find managers with at least five direct reports.
select e.name
from Employee e
join (select managerId
      from Employee
      group by managerId
      having count(managerId) >= 5) m on e.id = m.managerId
--# select name
--# from Employee
--# where id in (select managerId
--#              from Employee
--#              group by managerId
--#              having count(managerId) >= 5)

In [ ]:
-- M 1934. Confirmation Rate: find the confirmation rate of each user.
with noc as (select user_id, count(action) as num_of_confirmed
            from Confirmations
            where action = 'confirmed'
            group by user_id, action)
select s.user_id, ifnull(round(n.num_of_confirmed/count(c.user_id), 2), 0) as confirmation_rate
from Signups s
left join Confirmations c on s.user_id = c.user_id
left join noc n on c.user_id = n.user_id
group by s.user_id

--# {"headers": ["user_id", "num_of_action"], "values": [[3, 2], [7, 3], [2, 2], [6, 0]]}

--# Select s.user_id, ifnull(round((sum(c.action = 'confirmed')/count(c.action)), 2), 0) as confirmation_rate
--# From Signups s
--# left join Confirmations c on s.user_id = c.user_id
--# group by s.user_id;

--# Select s.user_id, ifnull(round(sum(case when c.action='confirmed' then 1 else 0 end)/count(c.user_id), 2), 0) as confirmation_rate
--# From Signups s
--# left join Confirmations c on s.user_id = c.user_id
--# group by s.user_id;

--# select s.user_id, round(avg(if(c.action="confirmed", 1, 0)), 2) as confirmation_rate
--# from Signups s
--# left join Confirmations c on s.user_id = c.user_id
--# group by user_id

In [ ]:
-- A 175. Combine Two Tables: report the first name, last name, city, and state of each person in the Person table.
SELECT p.firstName, p.lastName, a.city, a.state
FROM Person p
LEFT JOIN Address a ON (p.personId = a.personId)

In [ ]:
-- A 1607. Sellers With No Sales: report the names of all sellers who did not make any sales in 2020.
SELECT seller_name
FROM Seller
WHERE seller_id NOT IN (SELECT seller_id
                        FROM Orders
                        WHERE YEAR(sale_date) = 2020
                        GROUP BY seller_id)
ORDER BY seller_name

In [ ]:
-- A 1407. Top Travellers: result table ordered by travelled_distance in descending order, name in ascending order.
SELECT u.name, IFNULL(r.td, 0) AS travelled_distance
FROM Users u
LEFT JOIN (SELECT user_id, SUM(distance) AS td
           FROM Rides
           GROUP BY user_id) r
        ON u.id = r.user_id
ORDER BY travelled_distance DESC, u.name

In [ ]:
-- A 607. Sales Person: find the names of all the salespersons who did not have any orders related to the company with the name "RED".
SELECT name
FROM SalesPerson
WHERE sales_id NOT IN (SELECT o.sales_id
                       FROM Orders o, Company c
                       WHERE o.com_id = c.com_id and c.name = 'RED')